In [ ]:
import os
import sys

uniqname = "daltonri"
from google.colab import drive
drive.mount('/content/drive')
# TODO: Change this to the path to your homework folder
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/MoviePrediction'
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
['test.csv', 'train.csv', 'sample_submission.csv', 'Dataset_Investigation.ipynb']


In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

print(train.shape, test.shape)


(54000, 4) (36000, 4)


In [ ]:
train.head(30)

,id,movie_name,synopsis,genre
0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action
5,46636,Le démon dans l'île,"On an island, things start going crazy as the ...",horror
6,53777,Candid,A video voyeur stalks women in the city with a...,horror
7,24872,Hired,Twin brothers separated at birth and worlds ap...,crime
8,17224,Miruthan,A traffic police officer teams up with his fri...,adventure
9,54900,Until You See Me,A legendary tale unravels.,mystery


In [ ]:
test['genre'].value_counts()

action    36000
Name: genre, dtype: int64

In [ ]:
!pip install transformers
!pip install datasets
!pip install pip install transformers[torch]
!pip install evaluate
!pip install transformers-interpret

from datasets import Dataset
from transformers import AutoTokenizer, DistilBertModel
import torchtext
import pandas as pd

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.7 MB/s eta 0:00:00


In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = DistilBertModel.from_pretrained(model_name)

## Prepare Dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('train.csv')
genre = data['genre'].tolist()
synop = data['synopsis'].tolist()

In [ ]:
genreLabels = []
cnt = 0
GenreToLabel = dict()
for curGenre in genre:
  if curGenre not in GenreToLabel:
    GenreToLabel[curGenre] = cnt
    cnt = cnt + 1
  genreLabels.append(GenreToLabel[curGenre])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(synop, genreLabels, test_size=.2)

In [ ]:
def combineData(X,y):
  dataDict = []
  for idx in range(len(X)):
    dpoint = {'label': y[idx], 'text':X[idx]}
    dataDict.append(dpoint)
  return dataDict

In [ ]:
datasetDict = dict()
#datasetDict = combineData(synop, genreLabels)
datasetDict = {'label': genreLabels, 'text': synop}

dataset = Dataset.from_dict(datasetDict,split="train+test")
dataset = dataset.train_test_split(test_size=.2, shuffle=True)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Map:   0%|          | 0/43200 [00:00<?, ? examples/s]

Map:   0%|          | 0/10800 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=10)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.109859,0.213000
2,No log,1.917363,0.342000
3,No log,1.883728,0.362000


TrainOutput(global_step=375, training_loss=1.8513221028645834, metrics={'train_runtime': 370.99, 'train_samples_per_second': 8.086, 'train_steps_per_second': 1.011, 'total_flos': 789389862912000.0, 'train_loss': 1.8513221028645834, 'epoch': 3.0})

In [ ]:
from transformers_interpret import MultiLabelClassificationExplainer

index = 2035
synopsisToPredict = synop[index]
trueLabel = genre[index]

cls_explainer = MultiLabelClassificationExplainer(model, tokenizer)
word_attributions = cls_explainer(synopsisToPredict)

print(synopsisToPredict)
print(trueLabel)
print(GenreToLabel)
cls_explainer.visualize("multilabel_viz.html")

Ella has a reputation for two things: being a rebellious punk and the fastest gun around. When corrupt officials try to cover up her father's murder, she cuts a swath through the weird anachronistic underbelly of her small island town.
action
{'fantasy': 0, 'horror': 1, 'family': 2, 'scifi': 3, 'action': 4, 'crime': 5, 'adventure': 6, 'mystery': 7, 'romance': 8, 'thriller': 9}
